<a href="https://colab.research.google.com/github/xBrymer/FreeCodeCamp-Projects/blob/master/Machine-Learning/Book_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-requisites

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

import sklearn

Download the required files.

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

In [ ]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

Opening the data in a pandas data frame.

In [ ]:
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

Combining all the data.

In [ ]:
df_main = pd.merge(df_books, df_ratings, on = 'isbn', how = 'inner')

#Exploring the Data

In [ ]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_main.head()

,isbn,title,author,user,rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,8,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,11400,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,11676,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,41385,0.0


# Cleaning the Data


In [ ]:
#fil - filtered

fil_users = (df_ratings.
     groupby(by = ['user'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'total_rating_count'})
     [['user', 'total_rating_count']]
    )

USER_RATING_MIN = 200
fil_users.query('total_rating_count >= @USER_RATING_MIN',inplace=True)

In [ ]:
fil_books = (df_ratings.
     groupby(by = ['isbn'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'total_rating_count'})
     [['isbn', 'total_rating_count']]
    )

BOOK_RATING_MIN = 100
fil_books.query('total_rating_count >= @BOOK_RATING_MIN',inplace=True)

Exploring the Filtered Data

In [ ]:
fil_books.head()

,isbn,total_rating_count
3800,002542730X,171
5416,0060008032,104
5683,0060096195,107
6204,006016848X,147
6338,0060173289,130


In [ ]:
fil_users.head()

,user,total_rating_count
95,254,314
791,2276,498
981,2766,274
1049,2977,232
1177,3363,901


Merging the `fil_users` to `df_main` to get the ratings of the users.




In [ ]:
fil_main = pd.merge(fil_users, df_main, on = 'user', how = 'left')

In [ ]:
fil_main.drop('total_rating_count',1, inplace=True)

Merging the `fil_books` to `fil_main` to filter the books.


In [ ]:
df = pd.merge(fil_books, fil_main, on= 'isbn', how = 'left')

In [ ]:
df.drop('total_rating_count', 1, inplace = True)

Exploring the main dataframe

In [ ]:
df.head(10)

,isbn,user,title,author,rating
0,002542730X,3363.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,0.0
1,002542730X,11676.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,6.0
2,002542730X,12538.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,10.0
3,002542730X,13552.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,0.0
4,002542730X,16795.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,0.0
5,002542730X,24194.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,0.0
6,002542730X,25981.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,0.0
7,002542730X,26535.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,0.0
8,002542730X,28204.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,0.0
9,002542730X,30533.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,0.0


# Making the pivot table

In [ ]:
df_pivot = df.pivot_table(index = 'title', columns = 'user', values = 'rating').fillna(0)

Exploring the pivot table

In [ ]:
df_pivot.head()

user,254.0,2276.0,2766.0,2977.0,3363.0,4017.0,4385.0,6242.0,6251.0,6323.0,6543.0,6563.0,6575.0,7158.0,7286.0,7346.0,7915.0,8067.0,8245.0,8681.0,8936.0,9856.0,10447.0,10819.0,11601.0,11676.0,11993.0,12538.0,12824.0,12982.0,13082.0,13273.0,13552.0,13850.0,14422.0,14521.0,15408.0,15418.0,15957.0,16106.0,...,264317.0,264321.0,264637.0,265115.0,265313.0,265595.0,265889.0,266056.0,266226.0,266753.0,266865.0,266866.0,267635.0,268030.0,268032.0,268110.0,268330.0,268622.0,268932.0,269566.0,269719.0,269728.0,269890.0,270713.0,270820.0,271195.0,271284.0,271448.0,271705.0,273979.0,274004.0,274061.0,274301.0,274308.0,274808.0,275970.0,277427.0,277478.0,277639.0,278418.0
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Applying KNN

In [ ]:
matrix = csr_matrix(df_pivot)

In [ ]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'auto')
model_knn.fit(matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

# Testing the model

In [ ]:
def get_recommends(book = ''):

  distances, indices = model_knn.kneighbors(df_pivot.xs(book).values.reshape(1, -1), n_neighbors = 6)

  recommends = [book, []]
  for i in range(0, len(distances.flatten())):
    if i == 0:
        pass
    else:
        recommends[1].insert(0,[df_pivot.index[indices.flatten()[i]], distances.flatten()[i]]) 

  return recommends

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
